## The Saltenis estimator

In [ ]:
%matplotlib inline
from Functions import A1, A2, B1, B2, B3, C1, C2
import numpy as np
import pandas as pd
import sobol_seq
import matplotlib.pyplot as plt
from pandas import ExcelWriter
plt.style.use('ggplot')

### Defining the test functions and their analytical values

In [ ]:
k = 6

a2 = np.array([0,0.5,3,9,99,99])
b3 = np.array([6.42,6.42,6.42,6.42,6.42,6.42])

functions = [A1, A2, B1, B2, B3, C1, C2]

AE_df = pd.read_excel('AE_df.xlsx',index_col=0) #the analytical values

### It is then time to define the sample and scrambled matrices

In [ ]:
p = 13
run = 50

n = 2

df = pd.DataFrame(sobol_seq.i4_sobol_generate(2*k, run*2**p))

### And to assess the value of the MAE for the test functions

In [75]:
run_samples = []

MAE_dic = {f.__name__:pd.DataFrame(columns=[r for r in range(run)]) for f in functions}
for r in range (run):
    run_samples.append(df.iloc[int(r*(len(df)/run)):int((r+1)*(len(df)/run))].reset_index(drop=True))

    sample_Matrices = [run_samples[-1].iloc[:,m*int(len(run_samples[-1].T)/n):
                        (m+1)*int(len(run_samples[-1].T)/n)].T.reset_index(drop=True).T for m in range(n)]
    
    co = [1,0]
    mixed_Matrices = []
    for m in range(n):
        for j in range(k):
            mixed_Matrices.append(sample_Matrices[m].copy())
            mixed_Matrices[-1][j]=sample_Matrices[co[m]][j]
    
    elementary_effects_dic = {f.__name__:pd.DataFrame(columns=[j for j in range(k)]) for f in functions}
    elementary_effectsb_dic = {f.__name__:pd.DataFrame(columns=[j for j in range(k)]) for f in functions}
    for f in functions:
        for j in range(k):
            elementary_effects_dic[f.__name__][j] = 0.5*(f(sample_Matrices[0])-f(mixed_Matrices[j]))**2
            elementary_effectsb_dic[f.__name__][j] = 0.5*(f(sample_Matrices[1])-f(mixed_Matrices[j+k]))**2
    
        C_T = pd.concat([elementary_effects_dic[f.__name__],elementary_effectsb_dic[f.__name__]]).sort_index().expanding(1).mean()
        Var = f(pd.concat(sample_Matrices).sort_index()).expanding(1).var(ddof=0).expanding(1).mean()
        T = (C_T[~C_T.index.duplicated(keep='last')].T/Var[~Var.index.duplicated(keep='last')]).T
        
        AE_r = np.abs(T - AE_df.loc[f.__name__])
        MAE_r = AE_r.mean(axis=1)
        MAE_r.index=(MAE_r.index+1)*len(sample_Matrices)*(k+1)
        MAE_dic[f.__name__][r]=MAE_r

writer = pd.ExcelWriter('Saltenis_sym.xlsx', engine='xlsxwriter')

for mk in MAE_dic.keys():
    MAE_dic[mk].mean(axis=1).to_excel(writer, sheet_name=mk)
    
writer.save()